#Installations

In [ ]:
pip install texthero

     |████████████████████████████████| 241 kB 9.9 MB/s 
     |████████████████████████████████| 1.5 MB 39.0 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
pip install transformers

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 636 kB 62.4 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
     |████████████████████████████████| 3.3 MB 64.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.4 MB/s 


In [ ]:
pip install bert-serving-client

In [ ]:
pip install -U bert-serving-server[http]

     |████████████████████████████████| 61 kB 281 kB/s 
     |████████████████████████████████| 357 kB 17.9 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=612f1b900bda228bad0b1332d40235dd4172f98ca580adc1dc35a5438b63e0e5
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
#Download a model listed below, then uncompress the zip file into some folder, say /tmp/english_L-12_H-768_A-12/
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2021-07-29 19:48:27--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c06::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   151MB/s    in 2.6s    

2021-07-29 19:48:29 (151 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 -max_seq_len=512 &

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

#Import Libraries

In [ ]:
import texthero as hero
import pandas as pd
import difflib
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;

#Read Data

In [ ]:
df_old= pd.read_excel("AT&T_current_policy.xlsx")
df_new= pd.read_excel("AT&T_previous_policy.xlsx")

In [ ]:
df_old['clean_text'] = hero.clean(df_old['policy'])
df_new['clean_text'] = hero.clean(df_new['policy'])

#Create Embeddings

## BERT Embeddings

In [ ]:
#Find similarity using bert embeddings
df_new['Change_Score_BERT']=[None] * len(df_old['clean_text'])
for i in range(len(df_old['clean_text'])):
  sim=cosine_similarity(bc.encode([df_old['clean_text'][i]]),bc.encode([df_new['clean_text'][i]]))
  print(sim)
  df_new['Change_Score_BERT'][i]=1-sim

[[0.9995442]]
[[0.9783828]]
[[0.99387085]]
[[0.9082613]]
[[0.9967585]]


In [ ]:
df_new

,topic,policy,clean_text,Change_Score_BERT
0,Payment Terms Introduction,Services purchased using AutoPay are subject t...,services purchased using autopay subject prepa...,[[0.00045579672]]
1,Account Authorization,"By entering your checking account, credit card...",entering checking account credit card debit ca...,[[0.021617174]]
2,AutoPay,Account holders on a monthly plan will be noti...,account holders monthly plan notified via sms ...,[[0.0061291456]]
3,AutoPay details,This authorization will remain in effect until...,authorization remain effect cancel may cancel ...,[[0.0917387]]
4,ELECTRONIC COMMUNICATION CONSENT,By clicking Agree as Your electronic signature...,clicking agree electronic signature consent re...,[[0.0032414794]]


## Run for bert_ktrain

In [ ]:
df_xtrain_bert_embeddings= pd.read_excel("changedPolicy_AT&T_bert_and_ktrain.xlsx")
df_xtrain_bert_embeddings.head()

,Unnamed: 0,PolicyNumber,Severity,Question,Expected_PreviousPolicy_Answer,Expected_CurrentPolicy_Answer,Answer_PreviousPolicy,Answer_CurrentPolicy,Change_Score_TF_IDF,updated_Summary
0,0,0,2,Is the AutoPay amount refundable?,NaN,NaN,account balance is not refundable and is forfe...,account balance is not refundable and is forfe...,[[0.7211626]],Services purchased using AutoPay are subject t...
1,1,0,9,When will my account get cancelled?,NaN,NaN,accounts will be cancelled 60 days after expir...,accounts will be cancelled 60 days after expir...,[[0.18677966]],Services purchased using AutoPay are subject t...
2,2,1,8,What are different payment methods for AutoPay?,NaN,NaN,you agree to keep your payment method informat...,you agree to keep your payment method informat...,[[0.15428874]],Account holders on a monthly plan will be char...
3,3,1,1,Do you store payment method information?,NaN,NaN,you agree to keep your payment method informat...,you agree to keep your payment method informat...,[[0.15428874]],Account holders on a monthly plan will be char...
4,4,1,12,How often do I need to pay for monthly plans?,NaN,NaN,monthly plans may require a payment each month...,monthly plans may require a payment each month...,[[0.22402846]],Account holders on a monthly plan will be char...


In [ ]:
#Find similarity using bert embeddings
df_xtrain_bert_embeddings['Change_Score_BERT']=[None] * len(df_xtrain_bert_embeddings['Question'])
for i in range(len(df_xtrain_bert_embeddings['Question'])):
  sim=cosine_similarity(bc.encode([df_xtrain_bert_embeddings['Answer_PreviousPolicy'][i]]),bc.encode([df_xtrain_bert_embeddings['Answer_CurrentPolicy'][i]]))
  print(sim)
  df_xtrain_bert_embeddings['Change_Score_BERT'][i]=1-sim

In [ ]:
df_xtrain_bert_embeddings

,Unnamed: 0,PolicyNumber,Severity,Question,Expected_PreviousPolicy_Answer,Expected_CurrentPolicy_Answer,Answer_PreviousPolicy,Answer_CurrentPolicy,Change_Score_TF_IDF,updated_Summary,Change_Score_BERT
0,0,0,2,Is the AutoPay amount refundable?,NaN,NaN,account balance is not refundable and is forfe...,account balance is not refundable and is forfe...,[[0.7211626]],Services purchased using AutoPay are subject t...,[[-1.1920929e-07]]
1,1,0,9,When will my account get cancelled?,NaN,NaN,accounts will be cancelled 60 days after expir...,accounts will be cancelled 60 days after expir...,[[0.18677966]],Services purchased using AutoPay are subject t...,[[0.0]]
2,2,1,8,What are different payment methods for AutoPay?,NaN,NaN,you agree to keep your payment method informat...,you agree to keep your payment method informat...,[[0.15428874]],Account holders on a monthly plan will be char...,[[0.0]]
3,3,1,1,Do you store payment method information?,NaN,NaN,you agree to keep your payment method informat...,you agree to keep your payment method informat...,[[0.15428874]],Account holders on a monthly plan will be char...,[[0.0]]
4,4,1,12,How often do I need to pay for monthly plans?,NaN,NaN,monthly plans may require a payment each month...,monthly plans may require a payment each month...,[[0.22402846]],Account holders on a monthly plan will be char...,[[1.1920929e-07]]
5,5,1,7,I have a monthly plan. When will I be charged?,NaN,NaN,account holders on a monthly plan will be char...,account holders on a monthly plan will be char...,[[0.18691898]],Account holders on a monthly plan will be char...,[[-2.3841858e-07]]
6,6,2,4,I have a monthly plan. Do you send any notice ...,NaN,NaN,account holders on a monthly plan will be noti...,account holders on a monthly plan will be noti...,[[0.10865754]],Account holders on a monthly plan will be noti...,[[0.0]]
7,7,3,13,How will AutoPay cancellation affect?,NaN,NaN,auto pay cancellation or unsuccessful payment ...,auto pay cancellation or unsuccessful payment ...,[[0.40232214]],This authorization will remain in effect until...,[[0.0]]
8,8,3,3,Does AT&T keep my card information?,NaN,NaN,you agree to keep your payment method informat...,you agree to keep your payment method informat...,[[0.15428874]],This authorization will remain in effect until...,[[0.0]]
9,9,3,10,What will happen if I have insufficient funds ...,NaN,NaN,you can withdraw your consent at any time by c...,you can withdraw your consent at any time by c...,[[0.30445454]],This authorization will remain in effect until...,[[-1.1920929e-07]]


In [ ]:
df_xtrain_bert_embeddings.to_excel('changedPolicy_AT&T_bert_and_ktrain_new' +'.xlsx')

## TF-IDF Embeddings

In [ ]:
df_new['TF_IDF_PreviousPolicy'] = (
    hero.tfidf(df_old['clean_text'], max_features=50)
)

df_new['TF_IDF_CurrentPolicy'] = (
    hero.tfidf(df_new['clean_text'], max_features=50)
)

In [ ]:
df_new

,topic,policy,clean_text,Change_Score_BERT,TF_IDF_PreviousPolicy,TF_IDF_CurrentPolicy,Change_Score_TF_IDF
0,Payment Terms Introduction,Services purchased using AutoPay are subject t...,services purchased using autopay subject prepa...,[[0.00045579672]],"[0.12628605884559474, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.11065165787210039, 0.0, 0.0, 0.0, 0.0, 0.0,...",None
1,Account Authorization,"By entering your checking account, credit card...",entering checking account credit card debit ca...,[[0.021617174]],"[0.3452657575383885, 0.05863270474170112, 0.0,...","[0.4990191824444814, 0.07940243751288335, 0.0,...",None
2,AutoPay,Account holders on a monthly plan will be noti...,account holders monthly plan notified via sms ...,[[0.0061291456]],"[0.3990435631107133, 0.06776520087528641, 0.30...","[0.4228082425331125, 0.0, 0.2144233142550092, ...",None
3,AutoPay details,This authorization will remain in effect until...,authorization remain effect cancel may cancel ...,[[0.0917387]],"[0.0, 0.09163010430986986, 0.0, 0.091630104309...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.35668671975772626,...",None
4,ELECTRONIC COMMUNICATION CONSENT,By clicking Agree as Your electronic signature...,clicking agree electronic signature consent re...,[[0.0032414794]],"[0.18529654736247098, 0.0, 0.0, 0.146845461498...","[0.1606691331660769, 0.15339114713757002, 0.0,...",None


In [ ]:
#Find similarity using TF-IDF
df_new['Change_Score_TF_IDF']=[None] * len(df_old['clean_text'])
for i in range(len(df_old['clean_text'])):
  sim=cosine_similarity([df_new['TF_IDF_PreviousPolicy'][i]],[df_new['TF_IDF_CurrentPolicy'][i]])
  #print(sim)
  df_new['Change_Score_TF_IDF'][i]=1-sim

In [ ]:
df_new = df_new.drop(columns=['TF_IDF_PreviousPolicy', 'TF_IDF_CurrentPolicy'])
df_new

,topic,policy,clean_text,Change_Score_BERT,Change_Score_TF_IDF
0,Payment Terms Introduction,Services purchased using AutoPay are subject t...,services purchased using autopay subject prepa...,[[0.00045579672]],[[0.5236897835240916]]
1,Account Authorization,"By entering your checking account, credit card...",entering checking account credit card debit ca...,[[0.021617174]],[[0.25507163557277335]]
2,AutoPay,Account holders on a monthly plan will be noti...,account holders monthly plan notified via sms ...,[[0.0061291456]],[[0.3376216539816145]]
3,AutoPay details,This authorization will remain in effect until...,authorization remain effect cancel may cancel ...,[[0.0917387]],[[0.6308809735131118]]
4,ELECTRONIC COMMUNICATION CONSENT,By clicking Agree as Your electronic signature...,clicking agree electronic signature consent re...,[[0.0032414794]],[[0.37377421641562414]]


#Compare Rows

In [ ]:
def updated_policy(text1,text2):
  text1=text1.lower()
  text2=text2.lower()
  text1Lines=text1.split('.')
  text2Lines=text2.split('.') 
  
  diffInstance = difflib.Differ()
  diffList = list(diffInstance.compare(text1Lines, text2Lines))
  updated_lines=[]
  for line in diffList:
    if line[0] == '-':
      #print(line)
      updated_lines.append(line)
  updated_lines_joined = '.'.join(updated_lines)
  return updated_lines_joined


In [ ]:

df_new['updated']=[None] * len(df_old["policy"])
for i in range(len(df_old["policy"])):
  df_new['updated'][i]=updated_policy(df_new["policy"][i],df_old["policy"][i])

In [ ]:
df_new['omitted']=[None] * len(df_old["policy"])
for i in range(len(df_old["policy"])):
  df_new['omitted'][i]=updated_policy(df_old["policy"][i],df_new["policy"][i])

#Summarize differences

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

In [ ]:
def summ(text):
  input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True)
  summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=10,
                                  max_length=250,
                                  early_stopping=True)
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
  return summary

In [ ]:
df_new['updated_Summary']=[None] * len(df_old["policy"])
for i in range(len(df_old["policy"])):
  df_new['updated_Summary'][i]=summ(df_new["updated"][i])
  print(i)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning:

floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)



0
1
2
3
4


In [ ]:
df_new

,topic,policy,clean_text,Change_Score_BERT,Change_Score_TF_IDF,updated,omitted,updated_Summary
0,Payment Terms Introduction,Services purchased using AutoPay are subject t...,services purchased using autopay subject prepa...,[[0.00045579672]],[[0.5236897835240916]],"- this payment is not returnable, transferabl...","- payments are not returnable, transferable o...","This payment is not returnable, transferable o..."
1,Account Authorization,"By entering your checking account, credit card...",entering checking account credit card debit ca...,[[0.021617174]],[[0.25507163557277335]],"- by entering your checking account, credit ca...","- by entering your checking account, credit or...",You authorize at&t and/or its agents and your ...
2,AutoPay,Account holders on a monthly plan will be noti...,account holders monthly plan notified via sms ...,[[0.0061291456]],[[0.3376216539816145]],- account holders on a monthly plan will be no...,- account holders on a monthly plan will be no...,Account holders on a monthly plan will be noti...
3,AutoPay details,This authorization will remain in effect until...,authorization remain effect cancel may cancel ...,[[0.0917387]],[[0.6308809735131118]],- this authorization will remain in effect unt...,- this autopay authorization will remain in ef...,This authorization will remain in effect until...
4,ELECTRONIC COMMUNICATION CONSENT,By clicking Agree as Your electronic signature...,clicking agree electronic signature consent re...,[[0.0032414794]],[[0.37377421641562414]],- you can withdraw your consent at any time b...,- you can withdraw your consent at any time b...,You can withdraw your consent at any time by c...


In [ ]:
df_new.to_excel('changedPolicy_AT&T_diff_based_v2' +'.xlsx')